In [ ]:
import os
import cv2
import glob
import ntpath
import numpy
import pandas

In [ ]:
make_attribute_cube()

In [ ]:
def make_attribute_cube():
    
    src_path =  "G:/LungCancerPredict/extracted/lidc_extracted_label/"
    
    dst_path =  "G:/LungCancerPredict/extracted/lidc_extracted_image/"
    
    os.makedirs(dst_path, exist_ok=True)

    count=1
    
    for index, csv_file in enumerate(glob.glob(src_path + "*_nodule_attribute.csv")):
        
        patient_id = ntpath.basename(csv_file).replace("_nodule_attribute.csv", "") 
        
        df = pandas.read_csv(csv_file)
        
        if len(df) == 0: continue
        
        #读取该病人重采样后的图片
        imgs = load_patient_images(patient_id)
        
        for anno_index, row in df.iterrows():
            
            #读取nodule信息
            coord_x = int(row["coord_x"] * imgs.shape[2])
            coord_y = int(row["coord_y"] * imgs.shape[1])
            coord_z = int(row["coord_z"] * imgs.shape[0])
            diameter = row["diameter"] * imgs.shape[1]
            
            malignacy = row["malscore"];                    sphericiy = row["sphericiy"];   margin = row["margin"]
            spiculation = row["spiculation"];               texture = row["texture"];       calcification = row["calcification"]
            internal_structure = row["internal_structure"]; lobulation = row["lobulation"]; subtlety = row["subtlety"]
          
            #以坐标为中心创建一个64*64*64的cube
            cube_img = get_cube_from_img(imgs, coord_x, coord_y, coord_z, 64)
        
            #将该cube数据以8*8个，64*64的图片形式保存
            save_cube_img(dst_path + patient_id + "_" + str(anno_index) + "_" + str(diameter) + "_" + str(malignacy) + "_" + str(sphericiy) + "_" + str(margin) + "_" + str(spiculation) + "_" + str(texture) + "_" + str(calcification) + "_" + str(internal_structure) + "_" + str(lobulation) + "_" + str(subtlety) + "_1_pos.png", cube_img, 8, 8)

        print("Counter:",count)
        count+=1

def load_patient_images(patient_id):

    src_path = "G:/LungCancerPredict/extracted/luna16_extracted_images/" + patient_id + "/"
    src_files = glob.glob(src_path + "*_i.png")
    
    src_files.sort()
    
    imgs = [cv2.imread(src_file, cv2.IMREAD_GRAYSCALE) for src_file in src_files]
    imgs = [img.reshape((1, ) + img.shape) for img in imgs]
    
    res = numpy.vstack(imgs)
    
    return res

def get_cube_from_img(img3d, center_x, center_y, center_z, block_size):

    start_x = max(center_x - block_size / 2, 0) if center_x + block_size / 2 <= img3d.shape[2] else img3d.shape[2] - block_size
    start_y = max(center_y - block_size / 2, 0) if center_y + block_size / 2 <= img3d.shape[1] else img3d.shape[1] - block_size
    start_z = max(center_z - block_size / 2, 0) if center_z + block_size / 2 <= img3d.shape[0] else img3d.shape[0] - block_size

    start_z = int(start_z)
    start_y = int(start_y)
    start_x = int(start_x)
    
    res = img3d[start_z:start_z + block_size, start_y:start_y + block_size, start_x:start_x + block_size]
    
    return res

def save_cube_img(dst_path, cube_img, rows, cols):
    
    img_height = cube_img.shape[1]
    img_width = cube_img.shape[1]
    res_img = numpy.zeros((rows * img_height, cols * img_width), dtype=numpy.uint8)

    for row in range(rows):
        
        for col in range(cols):
            
            target_y = row * img_height
            target_x = col * img_width
            
            res_img[target_y:target_y + img_height, target_x:target_x + img_width] = cube_img[row * cols + col]

    cv2.imwrite(dst_path, res_img)